In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from typing import List
import pickle

In [5]:
with open("/data/s1620444/tm/data/bert_tokenized/bert_train_data.pkl", "rb") as f:
    train_dev = pickle.load(f)

with open("/data/s1620444/tm/data/bert_tokenized/bert_test_data.pkl","rb") as f:
    test = pickle.load(f)

In [2]:
# # load data, split
# train_dev = pd.read_csv("data/downloaded/twitter-train-clean.tsv", names=["label","text"],encoding="utf-8",sep='\t').dropna()
# # train, dev = train_test_split(train_dev, test_size=0.2)
# test = pd.read_csv("data/downloaded/twitter-test-clean.tsv", names=["label","text"],encoding="utf-8",sep='\t').dropna()

In [6]:
def map_label(label):
    if label == "negative":
        return 0
    elif label =="neutral":
        return 1
    elif label =="positive":
        return 2

X_train_dev = train_dev.text.to_list()
y_train_dev = train_dev.label.apply(lambda x: map_label(x)).to_list()
# X_dev = dev.text.to_list()
# y_dev = dev.label.apply(lambda x: map_label(x)).to_list()
X_test = test.text.to_list()
y_test = test.label.apply(lambda x: map_label(x)).to_list()

In [7]:
# tf idf tokenization
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(binary=True)
X_train_tfidf = tf_idf.fit_transform(X_train_dev)
# X_dev_tfidf = tf_idf.fit_tra?nsform(X_dev)

In [7]:
X_train_tfidf.shape

(40280, 42046)

In [8]:
from sklearn.linear_model import SGDClassifier
from scipy.sparse import csr_matrix 

In [9]:
def train_model(X: csr_matrix, y: List) -> None:
    X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2)
    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    dev_score= clf.score(X_dev, y_dev)
    print(f'Train score: {round(train_score,2)}, Dev score: {round(dev_score, 2)}')


In [10]:
train_model(X_train_tfidf, y_train_dev)

Train score: 0.73, Dev score: 0.66


In [11]:
# hyperparam tuning

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


# Phase 1: loss, learning rate and initial learning rate

clf = SGDClassifier()

distributions = dict(
    loss=['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    learning_rate=['optimal', 'invscaling', 'adaptive'],
    eta0=uniform(loc=1e-7, scale=1e-2)
)

random_search_cv = RandomizedSearchCV(
    estimator=clf,
    param_distributions=distributions,
    cv=5,
    n_iter=50
)
random_search_cv.fit(X_train_tfidf, y_train_dev)
print(f'Best params: {random_search_cv.best_params_}')
print(f'Best score: {random_search_cv.best_score_}')

/local/s1620444/tmprojectenv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/local/s1620444/tmprojectenv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/local/s1620444/tmprojectenv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/local/s1620444/tmprojectenv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/loc

Best params: {'eta0': 0.003985059891628318, 'learning_rate': 'adaptive', 'loss': 'squared_hinge'}
Best score: 0.6379592850049652


<h1> re-train best model with all data, predict </h1>

In [11]:
predictions = []

for i in range(3):
    best_clf = SGDClassifier(eta0 = 0.003985059891628318,learning_rate="adaptive", loss="squared_hinge",random_state=42)
    best_clf.fit(X_train_tfidf, y_train_dev)
    predictions.append(best_clf.predict(tf_idf.transform(X_test)))

In [18]:
with open("../results/baseline_predictions.pkl", "wb") as f:
    pickle.dump(predictions, f)
